In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

house_df_train_org = pd.read_csv('house_price_train.csv')
house_df_test_org = pd.read_csv('house_price_test.csv')
house_df_org = pd.concat([house_df_train_org,house_df_test_org])
house_df = house_df_org.copy()
house_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0


# EDA

In [2]:
#get dummies 실행
house_df_ohe = pd.get_dummies(house_df)

house_df_ohe.drop('Id', inplace=True, axis=1)
house_df_ohe['Bath'] = 2 * house_df_ohe['BsmtFullBath'] + house_df_ohe['BsmtHalfBath'] + 2 * house_df_ohe['FullBath'] + house_df_ohe['HalfBath']
house_df_ohe.drop('BsmtFullBath', inplace=True, axis=1)
house_df_ohe.drop('BsmtHalfBath', inplace=True, axis=1)
house_df_ohe.drop('FullBath', inplace=True, axis=1)
house_df_ohe.drop('HalfBath', inplace=True, axis=1)


#sales price가 맨 뒤에 오도록 함
sale_price_loc = house_df_ohe.columns.get_loc('SalePrice')
left = house_df_ohe.iloc[:, :sale_price_loc]
y_col = house_df_ohe.iloc[:, sale_price_loc]
right = house_df_ohe.iloc[:, sale_price_loc + 1:]

house_df_ohe = pd.concat([left, right, y_col], axis=1)
print(house_df_ohe.shape)
house_df_ohe.head(3)

(2919, 286)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,Bath,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,...,0,1,0,0,0,0,1,0,7.0,208500.0
1,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,...,0,1,0,0,0,0,1,0,5.0,181500.0
2,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,...,0,1,0,0,0,0,1,0,7.0,223500.0


In [3]:
#결측치 확인 및 처리
def nullProcEDA(df):
    ret_df = df.copy()
    
    for col in ret_df.columns[:-1]:
        tot_cnt = ret_df.shape[0]
        na_cnt = ret_df[col].isna().sum()
        na_rate = na_cnt / tot_cnt
        
        if na_rate > 0.0:
            print(f'{col:<20} 칼럼의 null 비율: {na_rate * 100:.2f}')
            mean = ret_df[col].mean()
            ret_df[col].fillna(mean, inplace=True)
        
    return ret_df

In [4]:
house_df_null = nullProcEDA(house_df_ohe)

LotFrontage          칼럼의 null 비율: 16.65
MasVnrArea           칼럼의 null 비율: 0.79
BsmtFinSF1           칼럼의 null 비율: 0.03
BsmtFinSF2           칼럼의 null 비율: 0.03
BsmtUnfSF            칼럼의 null 비율: 0.03
TotalBsmtSF          칼럼의 null 비율: 0.03
GarageYrBlt          칼럼의 null 비율: 5.45
GarageCars           칼럼의 null 비율: 0.03
GarageArea           칼럼의 null 비율: 0.03
Bath                 칼럼의 null 비율: 0.07


In [5]:
#데이터의 상관계수 분석

#일정 Threshold 이상의 correlation을 가지면 drop.
def correlationEDA(df, corr_thres = 0.75):
    ret_df = df.copy()
    
    corr_matrix = df.iloc[:, :-1].corr(method='pearson')
    
    checked = []
    dropped = []
    
    for col in corr_matrix.columns:
        high_corr = corr_matrix[(corr_matrix[col] <= -corr_thres) | (corr_matrix[col] >= corr_thres)].index
        checked.append(col)
        for ind in high_corr:
            if not (ind in checked) :
                checked.append(ind)
                dropped.append(ind)
                ret_df.drop(ind, inplace=True, axis=1)
                #print(f'Dropped Column: {ind}')
                print(f'Correlation of {col} and {ind}: {corr_matrix[col][ind] : .3f}')
    print(f'\nTotal {len(dropped)} Columns Dropped: \n{", ".join(dropped)}')
    return ret_df

In [27]:
house_df_corr = correlationEDA(house_df_null, corr_thres = 0.75)

Correlation of MSSubClass and BldgType_1Fam: -0.814
Correlation of YearBuilt and GarageYrBlt:  0.790
Correlation of TotalBsmtSF and 1stFlrSF:  0.802
Correlation of 2ndFlrSF and HouseStyle_1Story: -0.790
Correlation of 2ndFlrSF and HouseStyle_2Story:  0.810
Correlation of GrLivArea and TotRmsAbvGrd:  0.808
Correlation of KitchenAbvGr and BldgType_Duplex:  0.751
Correlation of GarageCars and GarageArea:  0.890
Correlation of MSZoning_FV and Neighborhood_Somerst:  0.867
Correlation of MSZoning_RL and MSZoning_RM: -0.805
Correlation of Street_Grvl and Street_Pave: -1.000
Correlation of LotShape_IR1 and LotShape_Reg: -0.933
Correlation of LotConfig_Corner and LotConfig_Inside: -0.759
Correlation of LandSlope_Gtl and LandSlope_Mod: -0.939
Correlation of Neighborhood_NPkVill and Exterior2nd_Brk Cmn:  0.799
Correlation of RoofStyle_Flat and RoofMatl_Tar&Grv:  0.791
Correlation of RoofStyle_Gable and RoofStyle_Hip: -0.939
Correlation of Exterior1st_AsbShng and Exterior2nd_AsbShng:  0.854
Correl

In [28]:
from sklearn.preprocessing import StandardScaler

# 각 column의 data distribution도 확인하고 싶지만 column이 너무 많아서 어떻게 확인할지 몰라 일단 통채로 standard scaler에 돌렸는데
# 그렇게 하니까 모든 평가지표가 굉장히 안좋게 나옵니다 어떻게 해결하는지는 잘 모르겠습니다ㅠㅠ
def scalingEDA(df):
    scaled_df = df.copy()
    scaler = StandardScaler()
    
    features = scaled_df.columns[:-1]
    scaled = scaler.fit_transform(scaled_df.iloc[:, :-1])
    Y = scaled_df.iloc[:,-1]
    Y = Y.reset_index(drop=True)
    scaled_data = pd.DataFrame(scaled, columns = features)
    
    scaled_df = pd.concat([scaled_data, Y], axis=1)
    
    return scaled_df

In [29]:
house_df_scaled = scalingEDA(house_df_corr)
house_df_scaled.tail()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,Bath,SalePrice
2914,2.419700,-2.266952,-1.043937,-1.481920,1.289758,-0.043346,-0.682812,-0.572250,-0.969192,-0.29313,...,-0.052423,-0.298629,-0.049029,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.889035,NaN
2915,2.419700,-2.266952,-1.049263,-1.481920,-0.507284,-0.043346,-0.682812,-0.572250,-0.415899,-0.29313,...,-0.052423,-0.298629,-0.049029,3.789876,-0.064249,-0.09105,-0.126535,-2.155466,-0.889035,NaN
2916,-0.873616,4.256207,1.246808,-0.772552,1.289758,-0.373528,0.561757,-0.572250,1.718232,-0.29313,...,-0.052423,-0.298629,-0.049029,3.789876,-0.064249,-0.09105,-0.126535,-2.155466,-0.270253,NaN
2917,0.655424,-0.342855,0.034605,-0.772552,-0.507284,0.683057,0.370284,-0.572250,-0.229272,-0.29313,...,-0.052423,-0.298629,-0.049029,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,-0.889035,NaN
2918,0.067331,0.220295,-0.068620,0.646183,-0.507284,0.716075,0.466021,-0.045921,0.695078,-0.29313,...,-0.052423,-0.298629,-0.049029,-0.263861,-0.064249,-0.09105,-0.126535,0.463937,0.348529,NaN


In [30]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test , pred)
    rmse = np.sqrt(mse)
    print('{0} 로그 변환된 RMSE: {1}'.format(model.__class__.__name__,np.round(rmse, 3)))
    return rmse

def get_rmses(models):
    rmses = [ ]
    for model in models:
        rmse = get_rmse(model)
        rmses.append(rmse)
    return rmses

## Model training

In [31]:
def trainTestDivider(df):
    train_set = df[~df['SalePrice'].isnull()]
    test_set = df[df['SalePrice'].isnull()]
    test_set.drop('SalePrice', inplace=True, axis=1)
    
    return train_set, test_set

train_set, test_set = trainTestDivider(house_df_corr)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error , r2_score

def trainEvaluate(df, test_df):
    y_target = df.iloc[:, -1]
    X_data = df.iloc[:, :-1]

    X_train , X_test , y_train , y_test = train_test_split(X_data , y_target ,test_size=0.2, random_state=156)

    lr = LinearRegression()
    lr.fit(X_train ,y_train)
    y_preds = lr.predict(X_test)
    mse = mean_squared_error(y_test, y_preds)
    rmse = np.sqrt(mse)

    print('MSE : {0:.3f} , RMSE : {1:.3F}'.format(mse , rmse))
    print('Variance score : {0:.3f}'.format(r2_score(y_test, y_preds)))
    
    coeff = pd.Series(data=np.round(lr.coef_, 1), index=X_data.columns )
    coeff.sort_values(ascending=False, inplace=True)
    print(coeff)
    
    y_test_preds = np.round(lr.predict(test_df), 0)
    y_df = pd.DataFrame(data=y_test_preds, columns = ['SalePrice'])
    
    return pd.concat([test_df, y_df], axis=1)

In [41]:
expected_price_df = trainEvaluate(train_set, test_set)

MSE : 833981105.068 , RMSE : 28878.731
Variance score : 0.876
RoofMatl_WdShngl     84665.3
GarageQual_Ex        82165.3
RoofMatl_Membran     78415.4
RoofStyle_Shed       69702.6
Condition2_RRNn      52494.3
                      ...   
GarageCond_Ex       -73870.6
PoolQC_Fa          -107269.6
PoolQC_Ex          -133113.3
Condition2_PosN    -185318.9
RoofMatl_ClyTile   -650455.9
Length: 251, dtype: float64


In [34]:
expected_price_df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,Bath,SalePrice
0,20,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,0,0,0,1,2.0,105405.0
1,20,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,0,0,0,1,3.0,147802.0
2,60,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,0,0,0,1,5.0,182518.0
3,60,78.0,9978,6,6,1998,1998,20.0,602.0,0.0,...,0,0,0,0,0,0,0,1,5.0,195087.0
4,120,43.0,5005,8,5,1992,1992,0.0,263.0,0.0,...,0,0,0,0,0,0,0,1,4.0,207455.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,3.0,86566.0
1455,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,...,0,0,0,1,0,0,0,0,3.0,76488.0
1456,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,...,0,0,0,1,0,0,0,0,4.0,178500.0
1457,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,...,0,0,0,0,0,0,0,1,3.0,110233.0
